In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pymongo
import requests
import splinter
from splinter import Browser
import os
import pandas as pd
from selenium import webdriver
import re

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [4]:
#open splinter browser
browser.visit(url)

In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(browser.html, 'html.parser')

In [6]:
result1 = soup.find_all('div', class_='content_title')

In [7]:
news_title = []
# Loop through returned results for news headline

for result in result1:
    news_title.append(result.a.text)
    
news_title[0]

"NASA's Treasure Map for Water Ice on Mars"

In [8]:
result2 = soup.find_all('div', class_='article_teaser_body')

In [9]:
news_p = []
# Loop through returned results for news paragraph

for result in result2:
    news_p.append(result.text)
    
news_p[0]

'A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.'

In [10]:
#JPL Mars Space Image

url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
soup = bs(browser.html, 'html.parser')
#find tag with featured image
div_tag = soup.find('a', class_='button fancybox')

#find attribute (data-link) that contains link to featured image
#div_tag['data-link']

#create link to access featured image
image_link = "https://jpl.nasa.gov" + str(div_tag['data-link'])
image_link



'https://jpl.nasa.gov/spaceimages/details.php?id=PIA19346'

In [11]:
#mars JPL image
url3 = image_link
browser.visit(url3)
soup = bs(browser.html, 'html.parser')

figure = soup.find_all('figure')
figure
image = soup.figure.a
#image['href']

featured_image_url = "https://jpl.nasa.gov" + str(image['href'])
featured_image_url

'https://jpl.nasa.gov/spaceimages/images/largesize/PIA19346_hires.jpg'

In [12]:
#mars twitter weather
url4 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url4)
soup = bs(browser.html, 'html.parser')

mars_tweet = soup.find('div', class_="js-tweet-text-container")
mars_weather = mars_tweet.p.text
mars_weather

'InSight sol 373 (2019-12-14) low -98.1ºC (-144.6ºF) high -20.1ºC (-4.1ºF)\nwinds from the SW at 4.9 m/s (11.1 mph) gusting to 19.7 m/s (44.0 mph)\npressure at 6.60 hPapic.twitter.com/J9DKptw2oo'

In [3]:
#parse mars facts with soup
url5 = 'https://space-facts.com/mars/'
browser.visit(url5)
soup = bs(browser.html, 'html.parser')

#mars_facts = soup.find(id='facts').strings

#for string in mars_facts:
#    print(repr(string))

In [4]:
#create a panda dataframe of the mars facts data and write it into a table in HTML
tables = pd.read_html(url5)
df = tables[0]
df.columns = ['description', 'value']
df.head()

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [5]:
#write pandas table to an html

html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [16]:
#visit https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
browser.visit('https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [18]:
#navigate and click on each link to each hemisphere
hemisphere =[]
hemisphere_list=['Cerberus','Schiaparelli','Syrtis','Valles']
for hemi in hemisphere_list:
    hemispheres={}
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    html = browser.html
    soup = bs(html,'html.parser')
    browser.click_link_by_partial_text(hemi)
    html = browser.html
    soup = bs(html,'html.parser')
    hemispheres['image']=soup.find('a',target="_blank")['href']
    hemispheres['title']=soup.find('h2',class_="title").text
    hemisphere.append(hemispheres)

print(hemisphere)

[{'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]
